# AnyoneAI - Project I - JHOEEL LUNA
# An analysis of NBA players in the 2021/2022 season

##1. Collecting information for building our dataset

1.1- Create a function to find all ACTIVE players, meaning players that are listed with a team in the 2021/2022 season. For now you only need the player id, name, and team. Save the dataframe to a csv named "nba_current_players_list.csv". The function should return the dataframe.

Hint: you should find an API method that can give you a list of players in just one call, this way we can filter those players we're interested in, and later will make calls for each specific player.

Consider dropping:
- All players with TEAM_ID == 0
- All players with GAMES_PLAYED_FLAG == N
- Players with id 1630597 & 1630492 (These guys are a problem ;))

In [ ]:
%pip install nba_api

In [ ]:
import pandas as pd
import numpy as np
import time
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
### Complete in this cell: get all active players from the api
def get_and_save_players_list():
  """This function fetchs the players list from the 'commonallplayers' endpoint

  :return: Pandas Dataframe
  """

  from nba_api.stats.endpoints import commonallplayers

  all_players = commonallplayers.CommonAllPlayers()
  df = all_players.common_all_players.get_data_frame()
  # Drops players without a team, no games played flag and players with id 1630597 and id 1630492
  df = df[(df.TEAM_ID != 0) & (df.GAMES_PLAYED_FLAG != 'N') & (df.PERSON_ID != 1630597) & (df.PERSON_ID != 1630492)]
  # resets index
  df = df.reset_index(drop=True)

  return df

In [ ]:
current_players_list = get_and_save_players_list()
current_players_list.to_csv("nba_current_players_list.csv")

*Don't change anything in this cell, just make it run correctly*

In [ ]:
if current_players_list.shape == (503, 16):
    print('Success!')
else:
    raise ValueError('The shape is incorrect, please review your function')

Success!


1.2- Create a function to find the personal information of all players listed in the dataframe created in the previous step, and save it to a csv file named "nba_players_personal_info.csv". The function should also return the created dataframe.

OPTIONAL: iterating on a list of players and making API calls can be complex and full of errors, try a code block that handles exceptions (for example a timeout from the API) and returns the partial result before it failed, you could also save the partial information to disk.

In [ ]:
### Complete in this cell: Find players personal information (name, age, dob, etc), store the information in a CSV file.
def get_players_personal_information(aList):
  """This function fetchs the players list from the 'commonplayerinfo' endpoint using a listed players in a dataframe

    :return: Pandas Dataframe
  """

  aList = current_players_list['PERSON_ID'].tolist()

  from nba_api.stats.endpoints import commonplayerinfo

  frames = []

  # The code below iterates over the list of players in order to make API calls of each players personal information
  for element in aList:
    players_info = commonplayerinfo.CommonPlayerInfo(player_id=element)
    df = players_info.get_data_frames()[0]
    frames.append(df)
    result = pd.concat(frames)
    # The statement below suspends the execution for the given number of seconds
    time.sleep(0.4)

  return result.reset_index(drop=True)

In [ ]:
players_personal_info = get_players_personal_information(current_players_list)
players_personal_info.to_csv("nba_players_personal_info.csv")

Don't change anything in this cell, just make it run correctly

In [ ]:
if players_personal_info.shape == (503, 33):
    print('Success!')
else:
    raise ValueError('The shape is incorrect, please review your function')

1.3- Create a function to find players career statistics, store the information in a CSV file called "nba_players_career_stats.csv"

In [ ]:
### Complete in this cell: find players career stats, save to csv file
def get_players_career_stats(current_players_list):
  """This function fetchs the players list from the 'playercareerstats' endpoint using a listed players in a dataframe

    :return: Pandas Dataframe
  """

  aList = current_players_list['PERSON_ID'].tolist()

  from nba_api.stats.endpoints import playercareerstats

  frames = []

  # The code below iterates over the list of players in order to make API calls of each players statistics
  for element in aList:
    players_stats = playercareerstats.PlayerCareerStats(player_id=element)
    df = players_stats.get_data_frames()[1]
    frames.append(df)
    result = pd.concat(frames)
    # The statement below suspends the execution for the given number of seconds
    time.sleep(0.4)

  return result.reset_index(drop=True)

In [ ]:
players_career_stats = get_players_career_stats(current_players_list)
players_career_stats.to_csv("nba_players_career_stats.csv")

Don't change anything in this cell, just make it run correctly

In [ ]:
if players_career_stats.shape == (503, 24):
    print('Success!')
else:
    raise ValueError('The shape is incorrect, please review your function')

1.4- Create a function to find players salary for this season. You can get the list of players and salaries from this file: [contracts.csv](https://drive.google.com/file/d/1bxjuIZfMS2_kickukdC3ggdXIVeZE5y7/view?usp=sharing). Download and use this file as input for your function `get_nba_players_salaries()`.

Also, inside this function, save the information to a csv called "nba_players_salary.csv". Make sure the players names format match the ones in the API, otherwise you won't be able to merge the data later.

Hint: Using data from the Basketball Reference page, you will have to solve 3 kinds of problems:

1.   Special characters that need to be removed.
2.   Duplicated values (for which you should keep just the first value) and,
3.   Players names not matching with the ones from the API.

The latter problem has multiple causes, one of them is that some names are written with non ascii characters, we advice you to install and use Unidecode library to solve this issue.

In [ ]:
### Complete in this cell: find players salary, save the information to csv
def get_nba_players_salaries(csv_file_path):
    """This function fetchs the players list from the 'contracts.csv' file using a listed players in a dataframe
        and merges two dataframes with IDs as key values

    :return: Pandas Dataframe
    """

    from unidecode import unidecode

    # The statement below reads the .csv file
    df = pd.read_csv(csv_file_path, skiprows=1, usecols=['Player','2021-22'])
    # The statement below drop duplicate players, however keeps the first
    df = df.drop_duplicates(subset=['Player'], keep='first')
    # The statement below removes the string characters after the backslash in the whole column
    df['Player'] = df['Player'].str.split('\\', expand=True)[0]
    # The statement below removes the '$' symbol in the whole column
    df['2021-22'] = df['2021-22'].str.replace('$', '', regex=True)
    # The statement below renames the columns
    df.rename(columns = {'Player':'PLAYER_NAME', '2021-22':'SALARY'}, inplace = True)
    # The statement below fixes encoding for players names with special characters
    df['PLAYER_NAME'] = df['PLAYER_NAME'].apply(unidecode)

    # The code below merges the current_players_list PERSON_IDs (key) with the PLAYERS NAMEs
    current_players_list = get_and_save_players_list()
    current_players_list.rename(columns = {'DISPLAY_FIRST_LAST':'PLAYER_NAME'}, inplace = True)
    df = pd.merge(current_players_list, df, on='PLAYER_NAME', how='left')
    df = df[['PERSON_ID', 'PLAYER_NAME', 'SALARY']]

    return df.reset_index(drop=True)

In [ ]:
players_salaries = get_nba_players_salaries("contracts.csv")
players_salaries.to_csv("nba_players_salary.csv")

Don't change anything in this cell, just make it run correctly

In [ ]:
index_salary_1 = players_salaries[players_salaries['PLAYER_NAME']=='Kyrie Irving'].index.values[0]
index_salary_2 = players_salaries[players_salaries['PLAYER_NAME']=='Luke Kennard'].index.values[0]

if players_salaries.SALARY.loc[index_salary_1] == '34916200' and players_salaries.SALARY.loc[index_salary_2] == '12727273':
    print('Success!')
else:
    raise ValueError('The salaries are incorrect, please review your function')

1.5- Create a function to merge the created dataframes: players_personal_info, players_career_stats, players_next_game, players_salaries. For each dataframe, select only the subset of columns needed to create the dataset described in section "The Dataset"

    - Players info: "PLAYER_NAME", "TEAM_NAME", "POSITION", "HEIGHT", "WEIGHT", "COUNTRY", "BIRTHDATE", "SEASON_EXP", "DRAFT_NUMBER"
    - Players stats: "GP", "MIN", "PTS", "REB", "AST", "STL", "BLK"
    - Misc: "SALARY"

Save the result to a csv called "raw_nba_players_dataset.csv"

Hint: Before merging the data, you should make sure all four dataframes have the same length, are indexed by PERSON_ID and have the same keys

In [ ]:
### Complete in this cell: merge the dataframes

def merge_dataframes(players_personal_info, players_career_stats, players_salaries):
    """This function merges all dataframes using IDs as key values

    :return: Pandas Dataframe
    """
    # The code below merges to the left the players list that contains the IDs (keys) with the personal information dataframe
    current_players_list = get_and_save_players_list()
    info = pd.read_csv("nba_players_personal_info.csv")
    cpl_info = pd.merge(current_players_list, info, on='PERSON_ID', how='left')

    # The code below merges to the left the new dataframe with the players statistics dataframe
    stats = pd.read_csv("nba_players_career_stats.csv")
    stats.rename(columns = {'PLAYER_ID':'PERSON_ID'}, inplace = True)
    cpl_info_stats = pd.merge(cpl_info, stats, on='PERSON_ID', how='left')

    # The code below merges to the left the new dataframe with the salaries dataframe
    salaries = pd.read_csv("nba_players_salary.csv")
    cpl_info_stats_salaries = pd.merge(cpl_info_stats, salaries, on='PERSON_ID', how='left')

    # The statement below allows the full wide view of the columns as we have merged many columns
    pd.set_option('display.max_columns', None)

    # This last code selects the columns required for the project
    df = cpl_info_stats_salaries[['PERSON_ID', 'PLAYER_NAME', 'TEAM_NAME_y', 'POSITION', 'HEIGHT', 'WEIGHT', 'COUNTRY', 'BIRTHDATE', 'SEASON_EXP', 'DRAFT_NUMBER', 'GP', 'MIN', 'PTS', 'REB', 'AST', 'STL', 'BLK', 'SALARY']]
    df.columns = df.columns.str.replace('TEAM_NAME_y', 'TEAM_NAME')
    df = df.set_index('PERSON_ID')

    return df

In [ ]:
raw_players_dataset = merge_dataframes(players_personal_info, players_career_stats, players_salaries)
raw_players_dataset.to_csv("raw_nba_players_dataset.csv")

*Don't change anything in this cell, just make it run correctly*

In [ ]:
if raw_players_dataset.shape == (503, 17):
    print('Success!')
else:
    raise ValueError('The shape is incorrect, please review your function')

##2. Data Cleaning and Preprocessing

There are several steps that you will have to follow, depending on where you have collected the data some information might be missing.  

- Height and weight might need to be converted to the metric system
- Players that have no team assigned should be removed from the dataset
- Players with no contracts (meaning they don't have a salary defined) should be removed from the dataset
- If the "position" data is ambiguous (listed at multiple positions), use the first value
- If the player does not have height or weight data, use the average for its position as its value

2.1- Create a copy of your dataset named "working_df", remove all players with no teams or salary

In [ ]:
### Complete in this cell: copy the dataset and drop NaNs in team or salary
def copy_and_delete_nan(players_dataset):
    """This function copies the raw players dataset and removes de NaN values in TEAM and SALARY columns

    :return: Pandas Dataframe
    """
    df = pd.read_csv(players_dataset, index_col=0)
    # Next statement removes players without a team or withouth a salary
    df = df.dropna(subset=['TEAM_NAME', 'SALARY'])

    return df

In [ ]:
players_dataset = "raw_nba_players_dataset.csv"

In [ ]:
working_df = copy_and_delete_nan(players_dataset)

Don't change anything in this cell, just make it run correctly

In [ ]:
if working_df.shape == (428, 17):
    print('Success!')
else:
    raise ValueError('The shape is incorrect, please review your function')

2.2- Cast Salary and Birthday columns to its corresponding type (int, datetime

In [ ]:
### Complete in this cell: cast all columns to its type
def cast_columns(working_df):
    """This function converts the datatypes from the columns BIRTHDATE and SALARY

    :return: Pandas Dataframe
    """
    # The statement below converts Birthdate column type to datetime
    working_df['BIRTHDATE'] = working_df['BIRTHDATE'].astype(np.dtype('<M8[ns]'))
    # The statement below converts Salary column type to integer
    working_df['SALARY'] = working_df['SALARY'].astype('int64')

    return working_df

In [ ]:
cast_columns(working_df)

Don't change anything in this cell, just make it run correctly

In [ ]:
if working_df.SALARY.dtype == 'int64' and working_df.BIRTHDATE.dtype == '<M8[ns]':
    print('Success!')
else:
    raise ValueError('The types are incorrect, please review your function')

2.3- Create a function that converts the height column from height in feet and inches to centimeter

In [ ]:
### Complete in this cell: convert height column
def convert_height_column(working_df):
    """This function converts the height values from Imperial system to Metric system

    :return: Pandas Dataframe
    """
    # The statement below converts the Height column from height in feet and inches to height in centimeters
    working_df['HEIGHT'] = round(working_df['HEIGHT'].str.split(pat = '-', expand = True).astype(int).dot([30.48, 2.54]), 0).astype(int)

    return working_df

In [ ]:
convert_height_column(working_df)

*Don't change anything in this cell, just make it run correctly*

In [ ]:
index_df_1 = working_df[working_df['PLAYER_NAME']=='Kyrie Irving'].index.values[0]
index_df_2 = working_df[working_df['PLAYER_NAME']=='Luke Kennard'].index.values[0]

if working_df.HEIGHT.loc[index_df_1] == 188 and working_df.HEIGHT.loc[index_df_2] == 196:
    print('Success!')
else:
    raise ValueError('The values are incorrect, please review your function')

2.4- Create a function that converts the weight column from pounds to kilograms

In [ ]:
### Complete in this cell: convert weight column
def convert_weight_column(working_df):
    """This function converts the weight values from Imperial system to Metric system

    :return: Pandas Dataframe
    """
    # The statement below converts the Weight column from weight in pounds to kilos
    working_df['WEIGHT'] = round(working_df['WEIGHT'].astype(float) * 0.45359237, 0).astype(int)

    return working_df

In [ ]:
convert_weight_column(working_df)

*Don't change anything in this cell, just make it run correctly*

In [ ]:
if working_df.WEIGHT.loc[index_df_1] == 88 and working_df.WEIGHT.loc[index_df_2] == 93:
    print('Success!')
else:
    raise ValueError('The values are incorrect, please review your function')

2.5- Create a function that calculates the age in (years, months, days) and saves it in a new string column named "AGE", example: "22 years, 5 months, 25 days"

In [ ]:
### Complete in this cell: add age column
def add_age_column(working_df):
    """This function creates a columns that expresses the age as a string

    :return: Pandas Dataframe
    """

    from datetime import datetime, date
    from dateutil import relativedelta

    ages = list()
    # The code belows iterates over the BIRTHDATE colums to convert the values to a string
    for i in working_df['BIRTHDATE']:
        # The statement below assigns a variable to the current date
        current_date = datetime.date(datetime.now())

        # The code below removes the symbol '-' and converts the values to various strings
        year = int(i.strftime('%Y-%m-%d').split('-')[0])
        month = int(i.strftime('%Y-%m-%d').split('-')[1])
        day = int(i.strftime('%Y-%m-%d').split('-')[2])
        start_date = date(year, month, day)

        # The code below combines the new strings to a single one
        delta = relativedelta.relativedelta(current_date, start_date)
        ages.append(f'{delta.years} Years {delta.month} Months {delta.days} Days')

    working_df['AGE'] = ages

    return working_df

In [ ]:
add_age_column(working_df)

2.6- Create a function that takes care of the disambiguation of the "POSITION" column. Should replace all positions that are mixed with the first one listed.

In [ ]:
### Complete in this cell: disambiguation of the position column
def update_position(working_df):
    """This function selects the first players position listed

    :return: Pandas Dataframe
    """
    # The statement below removes the characters next to the symbol '-'
    working_df['POSITION'] = working_df['POSITION'].str.split(pat='-', expand=True)[0]

    return working_df

In [ ]:
update_position(working_df)

*Don't change anything in this cell, just make it run correctly*

In [ ]:
if working_df.POSITION.loc[index_df_1] == 'Guard' and working_df.POSITION.loc[index_df_2] == 'Guard':
    print('Success!')
else:
    raise ValueError('The values are incorrect, please review your function')

2.7- Review that the working dataset has all requested columns with its corresponding datatypes and save it as a csv with name "nba_players_processed_dataset.csv"

In [ ]:
working_df.to_csv("nba_players_processed_dataset.csv")

##3. Analyzing and Visualizing data

Now that we have the data, let's do some work

3.1- Calculate and print the following metrics:

    - General metrics:
        - Total number of players
        - Number of USA born players
        - Number of foreign players
        - Number of players per position
        - Number of players per team
        - Number of rookies (first year players)
    - Players description
        - Average player age (in years)
        - Youngest player age (as a string, like: "22 years, 5 months, 25 days")
        - Oldest player age (as a string, like: "40 years, 2 months, 3 days")
        - Min and Max players height
        - Average height of players per position
    - Contracts
        - Min and Max salary of all players
        - Mean and Median salary of all players

Bonus: if you can, calculate how many players retired between the end of the 2020-2021 season and the start of the 2021-22 season.

In [ ]:
### Complete in this cell: print general metrics

# The code below creates a dataframe to show the total count (number) of players
df = pd.read_csv("nba_players_processed_dataset.csv")

pd.DataFrame (
    {
        'Count': [len(pd.unique(df['PERSON_ID'])), len(df[df['COUNTRY'] == 'USA']), len(df[df['COUNTRY'] != 'USA'])]
    },
    index=[
        'Total Number of Players', 'Number of USA born players', 'Number of foreign players'
    ]
)

In [ ]:
# The code below creates a dataframe to show the total count (number) of players per position in which they play
pd.DataFrame (
    {
        'Number of Players per Position': [len(df[df['POSITION'] == 'Forward']), len(df[df['POSITION'] == 'Center']), len(df[df['POSITION'] == 'Guard'])]
    },
    index=[
        'Forward', 'Center', 'Guard'
    ]
)

In [ ]:
# The statement below displays the number of players per team
df['TEAM_NAME'].value_counts(ascending=False)

In [ ]:
# The code below creates a dataframe to show the total count (number) of players considered as rookies (players with a first contract with less than four years)
pd.DataFrame (
    {
        'Number of rookies': [len(df[df['SEASON_EXP'] <= 4])]
    },
    index=[
        'Quantity'
    ]
)

In [ ]:
# The statement below creates a variable for the birthdate column in the dataframe
df_age = df['BIRTHDATE']

# The code below creates a function to convert date to countable numbers
def age(born):
    """This function converts a given date to age in number

    :return: Pandas Dataframe
    """

    from datetime import datetime, date

    born = datetime.strptime(born, '%Y-%m-%d').date()
    today = date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

df_age['Age'] = df_age.apply(age)
df['AGE_IN_YEARS'] = df_age['Age']

# The code below creates a dataframe to show players description
pd.DataFrame (
    {
        'Players Description': [round(df['AGE_IN_YEARS'].mean(), 2), df['AGE_IN_YEARS'].min(), df['AGE_IN_YEARS'].max(), df['HEIGHT'].min(), df['HEIGHT'].max()]
    },
    index=[
        'Average player age (in years)', 'Youngest player age', 'Oldest player age', 'Min height', 'Max height'
    ]
)

In [ ]:
# The statement below creates a dataframe to show players average height per position
df.groupby("POSITION", as_index=False).HEIGHT.mean().sort_values('HEIGHT', ascending=False)

In [ ]:
### Complete in this cell: Contracts

# The code below creates a dataframe to show players basic statistical data regarding their salaries
df = pd.read_csv("nba_players_processed_dataset.csv")

pd.DataFrame (
    {
        'Contracts': [df['SALARY'].min(), df['SALARY'].max(), df['SALARY'].mean(), df['SALARY'].median()]
    },
    index=[
        'Min salary of all players', 'Max salary of all players', 'Mean salary of all players', 'Median salary of all players'
    ]
)

3.2- Plot the relationship between scoring (points per game) and salary of all players, the players positions should also be visible.

In [ ]:
# The code below creates two columns: points per game and salary in millions in order to graph their relationships
df['SALARY_IN_MILLIONS'] = df['SALARY']/1000000
df['POINTS_PER_GAME'] = df['PTS']/df['GP']
df['SALARY_IN_MILLIONS'] = df['SALARY_IN_MILLIONS'].round(decimals = 2)
df['POINTS_PER_GAME'] = df['POINTS_PER_GAME'].round(decimals = 2)


In [ ]:
### Complete in this cell: Relationship between scoring and salary (in millions of dollars)

# The statement below plots the relationship between points per game, salary in millions and the position for each player
sns.pairplot(df, x_vars='POINTS_PER_GAME', y_vars='SALARY_IN_MILLIONS', hue='POSITION', diag_kind='kde', height=6).set(title='PPS vs Salary (in Millions) per position');

3.3- Now plot assists-vs-salary and rebounding-vs-salary

In [ ]:
### Complete in this cell: plot assist-vs-salary, rebounding-vs-salary

# The statement below plots the relationship between total number of assists per season and salary in millions and the position for each player
sns.pairplot(df, x_vars='AST', y_vars='SALARY_IN_MILLIONS', diag_kind='kde', height=6).set(title='Assists vs Salary');

In [ ]:
# The statement below plots the relationship between total number of rebounds per season and salary in millions and the position for each player
sns.pairplot(df, x_vars='REB', y_vars='SALARY_IN_MILLIONS', diag_kind='kde', height=6).set(title='Rebounds vs Salary');

3.4- When NBA players enter the league, they have low value salaries during what is called their "rookie contract". This means that it doesn't matter how well the player performs, they can't have large salaries. This can distort our understanding of how much teams value each skill, as a player could score 50 points a game and still earn just a couple of millions. So, lets now plot points, assists and rebounding vs salary, but only for players that have more than 4 years of experience (the typical length of a rookie contract)

In [ ]:
### Complete in this cell: non rookie contracts
# The statement below creates an auxiliar dataframe with only non rookie players
non_rookie_df = df[df['SEASON_EXP'] > 4]

# The statement below plots the relationship between points per game, salary in millions and the position for each non rookie player
sns.pairplot(non_rookie_df, x_vars='POINTS_PER_GAME', y_vars='SALARY_IN_MILLIONS', hue='POSITION', diag_kind='kde', height=6).set(title='PPS vs Salary (in Millions) per position (non rookies)');

In [ ]:
# The statement below plots the relationship between total number of assists per season and salary in millions and the position for each non rookie player
sns.pairplot(non_rookie_df, x_vars='AST', y_vars='SALARY_IN_MILLIONS', diag_kind='kde', height=6).set(title='Assists vs Salary (in Millions) non rookies');

In [ ]:
# The statement below plots the relationship between total number of rebounds per season and salary in millions and the position for each non rookie player
sns.pairplot(non_rookie_df, x_vars='REB', y_vars='SALARY_IN_MILLIONS', diag_kind='kde', height=6).set(title='Rebounds vs Salary (in Millions) non rookies');

3.5- Plot the scoring average grouped by position, we want to be able to see median, quartiles, etc

In [ ]:
### Complete in this cell: Scoring average grouped by position

# The code below plots scoring averages per position
sns.boxplot( y=df['POSITION'], x=df['PTS']);
plt.show()

3.6- Plot the Height distribution of all players

In [ ]:
### Complete in this cell: height distribution

#The statement below plots the height distribution
sns.histplot(df['HEIGHT']).set(title='Height distribution');